In [1]:
import os
import cv2
import sys
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [2]:
root = "/home/yanglei/DataSets/DAIR-V2X/Rope3D-KITTI-v2/training"

In [26]:
def load_depth(idx):
    img_filename = os.path.join(root, "depth", "{:06d}".format(idx) + ".jpg")
    isexist = True
    disp_img = cv2.imread(img_filename, cv2.IMREAD_UNCHANGED)
    if disp_img is None:
        isexist = False
        disp_img = np.zeros((370, 1224))
    else:
        disp_img = disp_img.astype(np.float)
    # return disp_img / 256.0, isexist
    return disp_img, isexist

def load_lidar(file_path):
    pc = np.fromfile(str(file_path), dtype=np.float32).reshape([-1, 4])
    return pc

def load_calib(idx):
    calib_name = os.path.join(root, "calib", "{:06d}".format(idx) + ".txt")
    with open(calib_name, 'r') as csv_file:
            reader = csv.reader(csv_file, delimiter=' ')
            for line, row in enumerate(reader):
                if row[0] == 'P2:':
                    P2 = row[1:]
                    P2 = [float(i) for i in P2]
                    P2 = np.array(P2, dtype=np.float32).reshape(3, 4)
                    break
    return P2

def inverse_rigid_trans(Tr):
    """ Inverse a rigid body transform matrix (3x4 as [R|t])
        [R'|-R't; 0|1]
    """
    inv_Tr = np.zeros_like(Tr)  # 3x4
    inv_Tr[0:3, 0:3] = np.transpose(Tr[0:3, 0:3])
    inv_Tr[0:3, 3] = np.dot(-np.transpose(Tr[0:3, 0:3]), Tr[0:3, 3])
    return inv_Tr

def get_depth_pt3d(depth):
    pt3d = []
    for i in range(depth.shape[0]):
        for j in range(depth.shape[1]):
            if depth[i, j] > 25 and depth[i, j] < 50:
                pt3d.append([i, j, depth[i, j]])
    return np.array(pt3d)

class Calibration(object):
    def __init__(self, P):
        # Projection matrix from rect camera coord to image coord
        self.P = np.reshape(P[:3, :], [3, 4])
        
        # Rigid transform from Velodyne coord to reference camera coord
        self.V2C = np.zeros((3, 4))
        self.V2C = np.reshape(self.V2C, [3, 4])
        self.C2V = inverse_rigid_trans(self.V2C)
        
        # Rotation from reference camera coord to rect camera coord
        self.R0 = np.zeros((3, 3))
        self.R0 = np.reshape(self.R0, [3, 3])

        # Camera intrinsics and extrinsics
        self.c_u = self.P[0, 2]
        self.c_v = self.P[1, 2]
        self.f_u = self.P[0, 0]
        self.f_v = self.P[1, 1]
        self.b_x = self.P[0, 3] / (-self.f_u)  # relative
        self.b_y = self.P[1, 3] / (-self.f_v)
    
    def project_image_to_rect(self, uv_depth):
        """ Input: nx3 first two channels are uv, 3rd channel
                   is depth in rect camera coord.
            Output: nx3 points in rect camera coord.
        """
        n = uv_depth.shape[0]
        x = ((uv_depth[:, 0] - self.c_u) * uv_depth[:, 2]) / self.f_u + self.b_x
        y = ((uv_depth[:, 1] - self.c_v) * uv_depth[:, 2]) / self.f_v + self.b_y
        
        if isinstance(uv_depth, np.ndarray):
            pts_3d_rect = np.zeros((n, 3))
        else:
            # torch.Tensor or torch.cuda.Tensor
            pts_3d_rect = uv_depth.new(uv_depth.shape).zero_()

        pts_3d_rect[:, 0] = x
        pts_3d_rect[:, 1] = y
        pts_3d_rect[:, 2] = uv_depth[:, 2]

        return pts_3d_rect

    def get_calib(self):
        return self.P
    
    def project_depth_to_rect(self, depth, constraint_box=False):
        depth_pt3d = get_depth_pt3d(depth)
        print(depth_pt3d.shape)
        depth_UVDepth = np.zeros_like(depth_pt3d)
        depth_UVDepth[:, 0] = depth_pt3d[:, 1]
        depth_UVDepth[:, 1] = depth_pt3d[:, 0]
        depth_UVDepth[:, 2] = depth_pt3d[:, 2]
        # print("depth_pt3d:",depth_UVDepth.shape)
        depth_pc_rect = self.project_image_to_rect(depth_UVDepth)
        # print("dep_pc_velo:",depth_pc_velo.shape)
        if constraint_box:
            depth_box_fov_inds = (
                (depth_pc_velo[:, 0] < cbox[0][1])
                & (depth_pc_velo[:, 0] >= cbox[0][0])
                & (depth_pc_velo[:, 1] < cbox[1][1])
                & (depth_pc_velo[:, 1] >= cbox[1][0])
                & (depth_pc_velo[:, 2] < cbox[2][1])
                & (depth_pc_velo[:, 2] >= cbox[2][0])
            )
            depth_pc_rect = depth_pc_rect[depth_box_fov_inds]
        return depth_pc_rect

In [27]:
depth, _ = load_depth(10)
P2 = load_calib(10)
calib = Calibration(P2)
pc = calib.project_depth_to_rect(depth)

/home/yanglei/anaconda3/envs/RSU/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



(642783, 3)


In [28]:
df_tmp = pd.DataFrame(pc[:, :3], columns=["x", "y", "z"])
df_tmp["norm"] = np.sqrt(np.power(df_tmp[["x", "y", "z"]].values, 2).sum(axis=1))

scatter = go.Scatter3d(
    x=df_tmp["x"],
    y=df_tmp["y"],
    z=df_tmp["z"],
    mode="markers",
    marker=dict(size=1, color=df_tmp["norm"], opacity=0.8),
)
fig = go.Figure(data=[scatter])
fig.update_layout(scene_aspectmode="data")
fig.show()

In [15]:
lidar_path = "/home/yanglei/DataSets/KITTI/training/velodyne/000327.bin"
pc = load_lidar(lidar_path)
pc.shape

(125770, 4)

In [29]:
df_tmp = pd.DataFrame(pc[:, :3], columns=["x", "y", "z"])
df_tmp["norm"] = np.sqrt(np.power(df_tmp[["x", "y", "z"]].values, 2).sum(axis=1))

scatter = go.Scatter3d(
    x=df_tmp["x"],
    y=df_tmp["y"],
    z=df_tmp["z"],
    mode="markers",
    marker=dict(size=1, color=df_tmp["norm"], opacity=0.8),
)
fig = go.Figure(data=[scatter])
fig.update_layout(scene_aspectmode="data")
fig.show()